# 1. Import Python Packages

In [1]:
%matplotlib notebook
import numpy as np

import sys, os
import glob
import json
sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas')

# # sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )

import metatlas.metatlas_objects as metob
# from metatlas.helpers import mzmine_batch_tools as mzm
from metatlas.helpers import mzmine_helpers as mzm
from metatlas.helpers import dill2plots as dp
from metatlas.helpers import metatlas_get_data_helper_fun as ma_data
from metatlas.helpers import chromatograms_mp_plots as cp

import pandas as pd

from matplotlib import pyplot as plt
# import multiprocessing as mp

from distutils.util import strtobool

from ast import literal_eval
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
# pd.set_option("display.max_colwidth", 1000000)

('NERSC=', True)


In [ ]:
# files = metob.retrieve('Lcmsruns',experiment='20180615_JS_FlavonoidsRice',username='*')
# for f in files:
#     print(f.name)

In [ ]:
# groups = dp.select_groups_for_analysis(name = '%20180703%Diurnal2%EPC18%',
#                                        most_recent = True,
#                                        remove_empty = True,
#                                        include_list = [], exclude_list = ['InjBL', 'ISTD', 'ITSD', 'QC'])#['QC','Blank'])

# files = []
# for g in groups:
#     for f in g.items:
#         files.append(f)
# sorted([f.name for f in files])

In [3]:
# Go from dataframe to list of dictionaries
#pd.DataFrame(mzmine_things).T.to_dict().values()
# pd.DataFrame(mzmine_things).to_csv('/global/homes/b/bpb/Downloads/mzmine_tasks.csv',index=False)
# df = pd.read_csv('/global/homes/b/bpb/Downloads/mzmine_tasks.csv')
# df = dp.get_google_sheet(notebook_name='Untargeted Job Submission',sheet_name='Sheet1')
df = dp.get_google_sheet(notebook_name='Untargeted Job Submission',sheet_name='Sheet1')

# df.columns = df.iloc[0]
# df = df.iloc[1:]

# do ast.literal_eval on lists
literal_cols = ['file_filters','groups']
for col in literal_cols:
    df[col] = df[col].apply(literal_eval)

df['process'] = df['process'].astype(bool)
df['is_group'] = [bool(strtobool(a)) for a in df['is_group']]
# df['remove_isotopes'] = [bool(strtobool(a)) for a in df['remove_isotopes']]
# df['peak_with_msms'] = [bool(strtobool(a)) for a in df['peak_with_msms']]

numeric_columns = [u'max_peak_duration', u'max_rt', u'min_intensity',
                   u'min_peak_duration', u'min_peak_height', u'min_rt', u'min_sn_ratio',
                   u'mz_tolerance', u'noise_floor', u'peak_to_valley_ratio',
                   u'rt_timespan', u'rt_tol_multifile', u'rt_tol_perfile', u'sample_to_blank']

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce', axis=1)

mzmine_things = df.T.to_dict().values()

# Create job scripts in output folder and print

In [ ]:
# df = pd.read_csv('/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel_Spain_vs_Berkeley/20160824_and_20170808_C18_MP_Solar_negative.sbatch')
# for d in df['#!/bin/sh']:
#     print(d)

In [11]:
import glob as glob
status = []
for i,params in enumerate(mzmine_things):
    project_name = '%s_%s'%(params['basename'],params['polarity'])
    results_dir = os.path.join(params['basedir'],project_name)
    
    # see if final mzmine feature table exists.
    job_done_1 = os.path.isfile(os.path.join(params['basedir'],'intermediate_results','%s_%s.csv'%(params['basename'],params['polarity'])),)
    # if mzmine workspace exists store this so it can be removed from job script for reruns
    mzmine_things[i]['mzmine_done'] = job_done_1
    
    # see if sheets/peak_height.tab exists
    job_done_2 = False
    peak_height_file = os.path.join(results_dir,'sheets','peak_height.tab')
    identification_folder = os.path.join(results_dir,'identification')
    num_id_figures = 0
    if os.path.isfile(peak_height_file):
        with open(peak_height_file,'r') as fid:
            peaks = fid.read()
            num_peaks = len(peaks.split('\n'))-1
        num_id_figures = len(glob.glob(os.path.join(identification_folder,'*.pdf')))
        if num_peaks > 0:
            job_done_2 = True
    else:
        num_peaks = 0
    # if metatlas is done note this too, but these jobs shouldn't be rerun (since they are done)
    mzmine_things[i]['metatlas_done'] = job_done_2
    
    # see if the filtered mzmine workspace has been created
    job_done_3 = os.path.isfile(os.path.join(results_dir,'%s.mzmine'%project_name))
    # this won't change the job script but completes the book-keeping
    mzmine_things[i]['small_mzmine_done'] = job_done_3
    
    status.append({'0_basedir':params['basedir'].split('/')[-1],'1_job':'%s_%s'%(params['basename'],params['polarity']),
                   '2_features_done':job_done_1,
                  '3_sheets__and_msms_done':job_done_2,
                   '4_workspace_done':job_done_3,
                  '5_num_features':num_peaks,
                  '6_num_id_figures':num_id_figures})
pd.DataFrame(status).to_csv('/global/homes/b/bpb/Downloads/mzmine_jobs_and_status.csv',index=False)    
pd.DataFrame(status)

,0_basedir,1_job,2_features_done,3_sheets__and_msms_done,4_workspace_done,5_num_features,6_num_id_figures
0,20170919_KBL_C18_TD-DB_Brauni,20170919_KBL_C18_TD-DB_BrauniiLipids_positive,False,True,False,1002,923
1,20170919_KBL_C18_TD-DB_Brauni,20170919_KBL_C18_TD-DB_BrauniiLipids_negative,False,True,False,1002,924
2,20170919_KBL_C18_TD-DB_Brauni,20170914_KBL_C18Lipid_TD-DB_Braunii_positive,False,True,False,1002,937
3,20170919_KBL_C18_TD-DB_Brauni,20170914_KBL_C18Lipid_TD-DB_Braunii_negative,False,True,False,1002,908
4,20170728_KBL_ZM_Root,20170728_KBL_C18_PZ-KM_RootFnl_negative,False,False,False,0,0
5,RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo,20170901_KBL_C18_DP-PB_PoplarRhizo_media_positive,True,False,False,0,0
6,P_Simiae_plate_flip,20170428_KBL_C18_Psimiae_DiffSubs_positive,False,True,False,1002,859
7,P_Simiae_plate_flip,20170428_KBL_C18_Psimiae_DiffSubs_negative,False,True,False,755,606
8,20170919_KBL_C18_TD-DB_Brauni,20171208_KBL_HILIC_Braunii_final_positive,False,True,False,1002,1
9,20170919_KBL_C18_TD-DB_Brauni,20171208_KBL_HILIC_Braunii_final_negative,False,True,False,1002,1


In [5]:
new_mzmine_things=[]
print(len(mzmine_things))
for i,params in enumerate(mzmine_things):
    # see if final mzmine file exists.  this will happen even if peak_height.tab is missing
    if (not params['metatlas_done']) or (not params['small_mzmine_done']) or (not params['mzmine_done']):
        new_mzmine_things.append(params)
mzmine_things = new_mzmine_things
print(len(mzmine_things))

120
24


In [6]:
for m in pd.unique([m['basedir'] for m in mzmine_things]):
    print('rm -r %s'%m)

rm -r /project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni
rm -r /project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZM_Root
rm -r /project/projectdirs/metatlas/projects/jgi_projects/RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo
rm -r /project/projectdirs/metatlas/projects/jgi_projects/P_Simiae_plate_flip
rm -r /project/projectdirs/metatlas/projects/jgi_projects/20171207_KBL_AS_JC_C18_MCom_1to3mem
rm -r /project/projectdirs/metatlas/projects/jgi_projects/20171004_KBL_HILIC_Shade_MicCom_final
rm -r /project/projectdirs/metatlas/projects/jgi_projects/20171207_KBL_AS_Untargeted
rm -r /project/projectdirs/metatlas/projects/jgi_projects/20190401_JGI_Metabolomics_Bootcamp_ReversePhase
rm -r /project/projectdirs/metatlas/projects/jgi_projects/20190401_JGI_Metabolomics_Bootcamp_HILIC
rm -r /project/projectdirs/metatlas/projects/jgi_projects/20190318_JJ_YH_UCBNiyogi_MetabMAA_20190314_QE144_Ag68377-924_USHXG01160
rm -r /project/projectdirs/metatlas/pro

In [7]:
for m in pd.unique([m['basedir'] for m in mzmine_things]):
    print('chmod -R 770 %s'%m)
    print('chgrp -R metatlas %s'%m)

chmod -R 770 /project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni
chgrp -R metatlas /project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni
chmod -R 770 /project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZM_Root
chgrp -R metatlas /project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZM_Root
chmod -R 770 /project/projectdirs/metatlas/projects/jgi_projects/RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo
chgrp -R metatlas /project/projectdirs/metatlas/projects/jgi_projects/RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo
chmod -R 770 /project/projectdirs/metatlas/projects/jgi_projects/P_Simiae_plate_flip
chgrp -R metatlas /project/projectdirs/metatlas/projects/jgi_projects/P_Simiae_plate_flip
chmod -R 770 /project/projectdirs/metatlas/projects/jgi_projects/20171207_KBL_AS_JC_C18_MCom_1to3mem
chgrp -R metatlas /project/projectdirs/metatlas/projects/jgi_projects/20171207_KBL_AS_JC_C18_MCom_1to3mem
chmod -R 770 /p

In [8]:
# groups = ['20160825_KBL_C18_MP_Solar_Berk','20170728_KBL_C18_MP_Solar_Spain']
mzm = reload(mzm)
for i,m in enumerate(mzmine_things):
    mzmine_things[i]['files'] = mzm.get_files(m['groups'],m['filename_substring'],m['file_filters'],m['is_group'])
    print(m['basename'],m['polarity'],len(mzmine_things[i]['files']))

('20170919_KBL_C18_TD-DB_BrauniiLipids', 'positive', 38)
('20170919_KBL_C18_TD-DB_BrauniiLipids', 'negative', 38)
('20170914_KBL_C18Lipid_TD-DB_Braunii', 'positive', 38)
('20170914_KBL_C18Lipid_TD-DB_Braunii', 'negative', 38)
('20170728_KBL_C18_PZ-KM_RootFnl', 'negative', 34)
('20170901_KBL_C18_DP-PB_PoplarRhizo_media', 'positive', 108)
('20170428_KBL_C18_Psimiae_DiffSubs', 'positive', 108)
('20170428_KBL_C18_Psimiae_DiffSubs', 'negative', 108)
14
('20171208_KBL_HILIC_Braunii_final', 'positive', 85)
14
('20171208_KBL_HILIC_Braunii_final', 'negative', 85)
46
('20171207_KBL_AS_JC_C18_MCom_1to3mem', 'negative', 145)
46
('20171207_KBL_AS_JC_C18_MCom_1to3mem', 'positive', 164)
46
('20171004_KBL_HILIC_Shade_MicCom_final', 'negative', 191)
46
('20171004_KBL_HILIC_Shade_MicCom_final', 'positive', 191)
46
('20171207_KBL_AS_JC_C18_MCom_1to3mem', 'negative', 145)
46
('20171207_KBL_AS_JC_C18_MCom_1to3mem', 'positive', 164)
46
('20171004_KBL_HILIC_Shade_MicCom_final', 'negative', 191)
46
('20171004

In [ ]:
# def merge_spectra(row):
#     qspec = np.asarray(eval(row['msv_query_aligned'].replace('nan','np.nan')))
#     rspec = np.asarray(eval(row['msv_ref_aligned'].replace('nan','np.nan')))
#     if row['database'] == 'nist':
#         rspec[0] = np.nan
#     sspec = np.array([qspec,rspec])#,axis=)#],axis=0)
#     return sspec
        
# new_spec = []
# for i,row in df.iterrows():
#     new_spec.append(merge_spectra(row))

# col_name = '((mz,intensity),(mz_ref,intensity_ref))'
# df[col_name] = new_spec
# df[col_name] = df[col_name].apply(lambda a: a.tolist())
# df.drop(columns=['msv_query_aligned','msv_ref_aligned']).to_csv('/global/homes/b/bpb/Downloads/test_spec_hits.csv',index=None)
# df.head()

In [ ]:
# files = glob.glob('/project/projectdirs/metatlas/raw_data/akuftin/20180731_KBL_C18_BrightSeed_PlantEx/*spectral-hits*')
# for f in files:
#     f2 = f.replace('spectral-hits','spectral-hits-confidential')
#     os.rename(f,f2)

# files = glob.glob('/project/projectdirs/metatlas/raw_data/akuftin/20180731_KBL_C18_BrightSeed_PlantEx/*spectral-hits-confidential*')

# for f in files:
#     df = pd.read_csv(f,sep='\t')
#     df.loc[df['database']=='nist','msv_ref_aligned'] = ''
#     f2 = f.replace('spectral-hits-confidential','spectral-hits')
#     print(f2)
#     df.to_csv(f2,sep='\t', compression='gzip')
    

In [9]:
mzm = reload(mzm)
for i,params in enumerate(mzmine_things):
    #setup all the job scripts when job actually runs it will copy params to params-used
    job_script = mzm.create_job_script(params)
    
    #dump the parameters to a json file
    with open( os.path.join(params['basedir'],'logs','%s_%s_params.json'%(params['basename'],params['polarity'])), 'w') as fid:
        fid.write(json.dumps(params))

    if len(params['files'])>500:
        print('sbatch %s'%job_script).replace('.sbatch','_denovo.sbatch')
    else:
        print('sbatch %s'%job_script)


sbatch /project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni/job_scripts/20170919_KBL_C18_TD-DB_BrauniiLipids_positive.sbatch
sbatch /project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni/job_scripts/20170919_KBL_C18_TD-DB_BrauniiLipids_negative.sbatch
sbatch /project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni/job_scripts/20170914_KBL_C18Lipid_TD-DB_Braunii_positive.sbatch
sbatch /project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni/job_scripts/20170914_KBL_C18Lipid_TD-DB_Braunii_negative.sbatch
sbatch /project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZM_Root/job_scripts/20170728_KBL_C18_PZ-KM_RootFnl_negative.sbatch
sbatch /project/projectdirs/metatlas/projects/jgi_projects/RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo/job_scripts/20170901_KBL_C18_DP-PB_PoplarRhizo_media_positive.sbatch
sbatch /project/projectdirs/metatlas/projects/jgi_projects/P_Simiae_pla

In [ ]:
# %system cat /project/projectdirs/metatlas/projects/jgi_projects/20171207_KBL_AS_Untargeted/job_scripts/20171004_KBL_HILIC_Shade_MicCom_final_positive_denovo.sbatch

In [ ]:
# mzm = reload(mzm)
# for i,params in enumerate(mzmine_things):
#     #setup all the job scripts when job actually runs it will copy params to params-used
#     job_script = mzm.create_job_script(params)
#     #dump the parameters to a json file
#     with open( os.path.join(params['basedir'],'logs','%s_%s_params.json'%(params['basename'],params['polarity'])), 'w') as fid:
#         fid.write(json.dumps(params))
#     print('sbatch %s'%job_script)


# make a network

In [ ]:
# sp = reload(sp)

In [ ]:
# from metatlas.helpers import spectralprocessing as sp
# df = pd.read_csv('/project/projectdirs/metatlas/projects/jgi_projects/20160205_C18_Qex_KZ_IsolatesExudates/20160205_C18_Qex_KZ_IsolatesExudates_positive/atlas_export.csv')
# all_edges = []
# for i in range(20): #do the top 10 mass differences
#     e = sp.make_edges(df['mz'].values,
#                df['label'].values,
#                sp.mass_differences.loc[i,'mass'],
#               tolerance=20,
#                edge_label=sp.mass_differences.loc[i,'formula'])
#     all_edges.extend(e)
# G = sp.transform_network_type(all_edges,output_type='networkx',do_mst=True)
# len(G.nodes)
# sp.write_network_for_cytoscape(G,'/global/homes/b/bpb/Downloads/kate_pos_network.csv')

# Gather files for a specific mzmine task

In [ ]:
# import os
# import fnmatch

# def recursive_glob(rootdir='.', pattern='*'):
#     """Search recursively for files matching a specified pattern.

#     Adapted from http://stackoverflow.com/questions/2186525/use-a-glob-to-find-files-recursively-in-python
#     """
#     matches = []
#     for root, dirnames, filenames in os.walk(rootdir):
#         for filename in fnmatch.filter(filenames, pattern):
#             matches.append(os.path.join(root, filename))
#     return matches

# params_files = recursive_glob('/project/projectdirs/metatlas/projects/jgi_projects/20180502_KBL_C18_PT_SwtPotVeggies/',pattern='peak_height.tab')
# params_files

In [ ]:
# import simplejson as json

# for f in params_files:
#     obj = None
#     with open(f,'r') as fid:
#         obj = json.load(fid)

#     outfile = open(os.path.join('/global/homes/b/bpb/Downloads/',os.path.basename(f)), "w")
#     outfile.write(json.dumps(obj, indent=4, sort_keys=True))
#     outfile.close()




In [ ]:
# tar_str = 'tar -czvf pactolus_files.tar.gz '
# for i in df[df.basedir.str.contains('erbe')].index:
#     for f in mzmine_things[i]['files']:
#         print('cp %s ~/pactolus_files'%f.replace('.mzML','.pactolus.gz'))
# #         tar_str = '%s %s'%(tar_str,f.replace('.mzML','.pactolus.gz'))
# #         print(os.path.isfile(f.replace('.mzML','.pactolus.gz')))
# # print tar_str

In [ ]:


# %system cat /project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZerbeMurphy/20170728_KBL_C18_PZ-KM_RootFnl_positive.sbatch

In [ ]:
# [bpb@cori06 ~ 02:14 PM] ~ > salloc -N 1 -q interactive -C haswell -t 4:00:00 -L project
# salloc: Pending job allocation 10461042
# salloc: job 10461042 queued and waiting for resources
# salloc: job 10461042 has been allocated resources
# salloc: Granted job allocation 10461042
# salloc: Waiting for resource configuration
# salloc: Nodes nid00119 are ready for job
# [bpb@nid00119 ~ 02:14 PM] ~ > echo $HOSTNAME
# cori06

In [ ]:
# mzm = reload(mzm)
# mzm.clean_and_filter_mzmine_output(json_file)


In [ ]:
# # mzm = reload(mzm)

# # json_file = "/global/project/projectdirs/metatlas/projects/jgi_projects/20171207_KBL_AS_Untargeted/logs/20171004_KBL_HILIC_Shade_MicCom_final_negative_params-used.json"
# # json_file = "/global/project/projectdirs/metatlas/projects/jgi_projects/20180502_KBL_C18_PT_SwtPotVeggies_10k/logs/20180502_KBL_C18_PT_SwtPotVeggies_10k_positive_params-used.json"
# json_file = "/global/project/projectdirs/metatlas/projects/jgi_projects/20171207_KBL_AS_Untargeted/logs/20171004_KBL_HILIC_Shade_MicCom_final_positive_params-used.json"

# import multiprocessing as mp
# NUM_THREADS=12
# def clean_and_filter_mzmine_output(json_filename=None,n_peaks=1000,do_test=False):
#     """
#     This is a command line function that runs after the first feature finding is complete.
#     After mzmine runs, make an atlas, get data, and filter
#     The principle is to cast the mzmine peaks into three different categories:
#         blank
#         bad peak shape
#         good no msms
#         good with msms
#     First, Remove in blank:
#         This can be done from mzmine output
#         exclude peaks where neither peak height nor peak area is appreciably greater in sample vs blank
#     Second, Remove Bad Peak shape:
#         seems like rt_peak being near rt@max is a good thing
#         seems like looking at intensity max
#         you don't want to exclude peaks like leu-ileu.
#         many peaks will have multiple-humps
#         A simplified logic is: is max intensity near the rt_peak? if yes, is it decaying by 10x +/- 0.5 minutes. Obviously, peaks that are +/- 0.5 minutes appart will get removed. Thus, find local minimum +/- 0.5 minutes from rt_peak. Set those as rt_min and rt_max.
#     Third remove no MS/MS
#         exclude if no feature in non-blank samples has msms
        
#     Last: only keep n_peaks by max intensity in any sample.

#     """


#     if not json_filename:
#         json_filename = sys.argv[1]
#     with open(json_filename) as data_file:    
#         params = json.load(data_file)
#     file_to_convert = os.path.join(params['basedir'],'intermediate_results','%s_%s.csv'%(params['basename'],params['polarity']))
#     print('# Working on %s %s'%(params['basename'],params['polarity']))
#     if os.path.isfile(file_to_convert):
#         # take the comprehensive csv from mzmine and make a peak-height only version of it
#         df,original_mzmine = mzm.metatlas_formatted_atlas_from_mzmine_output(file_to_convert,params['polarity'],
#                                                                                  make_atlas=False,min_rt=0.55,
#                                                                                 remove_fragments=False,
#                                                                                 remove_adducts=False,
#                                                                                 remove_clusters=False,)
#         df = df[abs(df['mz']-229.9811)<1]
#         print(df.shape)
        
#         if do_test == True:
#             #if only a test, just do a couple of features
#             df = df.head(10)
            
# #         df.to_csv(file_to_convert.replace('.csv','') + '_formatted.csv',index=True) #save a simplified mzmine-like csv as a backup of all features found.
        
#         #Filter features found in the blank
#         df = mzm.clean_up_mzmine_dataframe(df)
        
#         df_blank_compare = df.transpose().groupby(['b' if any([s in g.lower() for s in params['blank_str']]) else 's' for g in df.columns]).max().transpose()
#         if 'b' in df_blank_compare.columns:
#             df_features_not_in_blank = df_blank_compare[df_blank_compare['s'] > (params['sample_to_blank'] * df_blank_compare['b'])]
#             print('# %d features total'%(df.shape[0]))
#             print('# %d features removed by blank\n'%(df_blank_compare.shape[0] - df_features_not_in_blank.shape[0]))
#         else:
#             print('# No files have "blank" or "injbl" in their names.')
#             df_features_not_in_blank = df_blank_compare
#         df_features_not_in_blank.reset_index(inplace=True)
#         print('#There are now %d features not in blank'%df_features_not_in_blank.shape[0])
        
        
#         #Make an Atlas
#         print('making new atlas')
#         cids = []
#         for j,row in df_features_not_in_blank.iterrows():
#             my_mz_ref = metob.MzReference(mz=row.mz,mz_tolerance=row.mz_tolerance,detected_polarity=params['polarity'],lcms_run=None)
#             use_rt = row.rt_min
#             if row.rt_min < params['min_rt']:
#                 use_rt = params['min_rt']
#             my_rt_ref = metob.RtReference(rt_peak=row.rt_peak,rt_min=use_rt,rt_max=row.rt_max,lcms_run=None)
#             my_id = metob.CompoundIdentification(rt_references=[my_rt_ref],mz_references=[my_mz_ref],name=row.label)
#             cids.append(my_id)
#         my_atlas = metob.Atlas(name='untargeted atlas',compound_identifications=cids)
#         atlas_df = ma_data.make_atlas_df(my_atlas)
        
#         #Make Groups
#         print('making groups')
#         all_files = [f.replace('Peak height','').replace('filtered','').strip() for f in df.columns if '.mzML' in f]
#         metatlas_files = []      
#         for f in all_files:
#             f = metob.retrieve('Lcmsruns',name=f,username='*')[-1]
#             if isinstance(f,type(metob.LcmsRun())):
#                 metatlas_files.append(f)
#             else:
#                 print('%s NOT FOUND'%f)
#                 break
#         groups = metob.Group(name='untargeted group',items=metatlas_files)
        
#         #Get Data
#         print('getting data')
#         print('using',NUM_THREADS,'cores')
#         all_files = []
#         for my_file in groups.items:
#             all_files.append((my_file,groups,atlas_df,my_atlas))
#         # print('trying it without multiprocessing')
#         # metatlas_dataset = []
#         # for f in all_files:
#             # metatlas_dataset.append(ma_data.get_data_for_atlas_df_and_file(f))

#         pool = mp.Pool(processes=min(NUM_THREADS, len(all_files)))
#         metatlas_dataset = pool.map(ma_data.get_data_for_atlas_df_and_file, all_files)
#         pool.close()
#         pool.join()
#         # pool.terminate()
#         print('data acquired')
#     return metatlas_dataset,atlas_df

# # FILTER IT AFTER GETTING FRESH DATA. THE DATA FROM THE MZMINE OUTPUT IS WRONG!

# metatlas_dataset,atlas_df = clean_and_filter_mzmine_output(json_file)

# # # corr = df.reset_index(drop=True).T.diff(periods=1)

# # # zero_data = df.reset_index(drop=True).fillna(0)
# # # distance = lambda column1, column2: pd.np.linalg.norm(column1 - column2)
# # # result = zero_data.apply(lambda col1: zero_data.apply(lambda col2: distance(col1, col2)))
# # # result.stack().reset_index()
# # # result.columns = ['var1', 'var2','value']

# # # links = corr.stack().reset_index()
# # # links.columns = ['var1', 'var2','value']
# # # # Keep only correlation over a threshold and remove self correlation (cor(A,A)=1)
# # # links_filtered=links.loc[(links['var1'] < links['var2']) ] # (links['value'] > 0.9) & 
# # # links_filtered
# # # df_intensity = pd.read_csv('/Users/bpb/Downloads/peak_height (13).tab',sep='\t',header=1)
# # # df_intensity.drop(columns=['file'],inplace=True)
# # # # Calculate the correlation between individuals. We have to transpose first, because the corr function calculate the pairwise correlations between columns.
# # # corr = df_intensity.T.corr()
# # # # Transform it in a links data frame (3 columns only):
# # # links = corr.stack().reset_index()
# # # links.columns = ['var1', 'var2','value']
# # # # Keep only correlation over a threshold and remove self correlation (cor(A,A)=1)
# # # links_filtered=links.loc[ (links['value'] > 0.8) & (links['var1'] != links['var2']) ]
# # # # Build your graph
# # # G = nx.from_pandas_edgelist(links_filtered, source='var1', target='var2',edge_attr='value')
# # # nx.write_edgelist(G, "/Users/bpb/Downloads/corr_edge_list.csv", data=['value'],delimiter=',')

# # # #.sort_values('rt_peak')
# # # #peak is at 8.45

In [ ]:
# json_file = "/global/project/projectdirs/metatlas/projects/jgi_projects/20171207_KBL_AS_Untargeted/logs/20171004_KBL_HILIC_Shade_MicCom_final_positive_params-used.json"

# import random

# def setup_run(json_filename=None,n_peaks=None,n_files=None):
#     if not json_filename:
#         json_filename = sys.argv[1]
#     with open(json_filename) as data_file:    
#         params = json.load(data_file)
#     file_to_convert = os.path.join(params['basedir'],'intermediate_results','%s_%s.csv'%(params['basename'],params['polarity']))
#     print('# Working on %s %s'%(params['basename'],params['polarity']))
#     if os.path.isfile(file_to_convert):
#         # take the comprehensive csv from mzmine and make a peak-height only version of it
#         df,original_mzmine = mzm.metatlas_formatted_atlas_from_mzmine_output(file_to_convert,params['polarity'],
#                                                                                  make_atlas=False,min_rt=0.55,
#                                                                                 remove_fragments=False,
#                                                                                 remove_adducts=False,
#                                                                                 remove_clusters=False,)
        
            
        
        
#         df = mzm.clean_up_mzmine_dataframe(df)
#         df.reset_index(inplace=True)
#         if n_peaks is not None:
#             df = df.sample(n=n_peaks)
        
#         #Make an Atlas
#         print('making new atlas')
#         cids = []
#         for j,row in df.iterrows():
#             my_mz_ref = metob.MzReference(mz=row.mz,mz_tolerance=row.mz_tolerance,detected_polarity=params['polarity'],lcms_run=None)
#             use_rt = row.rt_min
#             if row.rt_min < params['min_rt']:
#                 use_rt = params['min_rt']
#             my_rt_ref = metob.RtReference(rt_peak=row.rt_peak,rt_min=use_rt,rt_max=row.rt_max,lcms_run=None)
#             my_id = metob.CompoundIdentification(rt_references=[my_rt_ref],mz_references=[my_mz_ref],name=row.label)
#             cids.append(my_id)
#         my_atlas = metob.Atlas(name='untargeted atlas',compound_identifications=cids)
#         atlas_df = ma_data.make_atlas_df(my_atlas)
        
#         #Make Groups
#         print('making groups')
#         all_files = [f.replace('Peak height','').replace('filtered','').strip() for f in df.columns if '.mzML' in f]
#         metatlas_files = []
        
#         if n_files is not None:
#             random.shuffle(all_files)
#             all_files = all_files[:n_files]
        
#         for f in all_files:
#             f = metob.retrieve('Lcmsruns',name=f,username='*')[-1]
#             if isinstance(f,type(metob.LcmsRun())):
#                 metatlas_files.append(f)
#             else:
#                 print('%s NOT FOUND'%f)
#                 break
#         groups = metob.Group(name='untargeted group',items=metatlas_files)
        
#         run_dict = {'atlas':my_atlas,'atlas_df':atlas_df,'groups':groups}
        
#         return run_dict

# run_dict = setup_run(json_filename=json_file,n_peaks=None,n_files=10)


In [ ]:
# run_dict['atlas_df'].shape

In [ ]:
# %%time
# filename = run_dict['groups'].items[0].hdf5_file
# print(filename)
# msdata = ma_data.df_container_from_metatlas_file(filename)
# msdata = msdata['ms1_pos']

In [ ]:
# %%time
# from scipy import interpolate

# def trim_dataset_by_mz_ppm(atlas,msdata,ppm_cutoff):
#     mz_atlas = atlas['mz'].values[:]
#     mz_data = msdata['mz'].values[:]
#     f = interpolate.interp1d(mz_atlas, np.arange(len(mz_atlas)),kind='nearest',bounds_error=False,fill_value='extrapolate') #get indices of all mz values in the atlas
#     idx = f(mz_data)   # iterpolate to find the nearest mz in the data for each mz in an atlas
#     idx = idx.astype('int')
#     d = np.abs(mz_data - mz_atlas[idx]) / mz_data * 1.0e6
#     return msdata.loc[d<ppm_cutoff,:]

# msdata_filtered = trim_dataset_by_mz_ppm(run_dict['atlas_df'],msdata,10)


# # fill value of nan outside range is default
# # r = range(10)
# # r=np.asarray(r)+1000000
# # for i in r:
# #     print(mz_data[i],idx[i],d[i])
# #     print(mz_atlas[idx[i]])
# #     print('')
# # nearest_mz = mz_atlas[idx.astype('int8')]

# # temp = msdata[np.isclose(msdata['mz'].values[:, None], run_dict['atlas_df']['mz'].values[:100], atol=0.01,rtol=0).any(axis=1)]
# # print(msdata.shape,temp.shape)

# # only works in python3
# # https://stackoverflow.com/questions/41240067/pandas-and-multiprocessing-memory-management-splitting-a-dataframe-into-multipl

In [ ]:
# idx2 = f(mz_atlas).astype('int')
# len(idx2),len(list(set(idx2)))

In [ ]:
# float(msdata_filtered.shape[0])/float(msdata.shape[0])

In [ ]:
# %%time
# for i,row in run_dict['atlas_df'].iterrows():
# #     temp = ma_data.get_data_for_mzrt(row,msdata_filtered.loc[idx<100,:],None,extra_time=0)
#     temp = ma_data.get_data_for_mzrt(row,msdata.loc[idx==i,:],None,extra_time=0)
# #     temp = ma_data.get_data_for_mzrt(row,msdata,None,extra_time=0)


In [ ]:
# a little of 6 minutes is the time to beat. With the dataset about half as big, its about 3 minutes, using index from m/z it is 1.5 minutes.  needs to be checked for accuracy... it won't be right since mz from many features are only allowed to map to one feature

# you would need to identify natural breaks in m/z's from the atlas and break up the data and atlas using those breaks into very small pieces
# here is how to find the optimum number of breaks:
# https://stats.stackexchange.com/questions/143974/jenks-natural-breaks-in-python-how-to-find-the-optimum-number-of-breaks

# or just sort them and split if diff greater than threshold.  Then, bundle the groups into n groups.

# it seems like there must be a way to use the interp1d backwards and map features that have m/z within range together.

In [ ]:
# #indices of original data in feature
# temp[0].index.tolist() #probably use this

# #dataframe of values for feature
# temp[1].T.shape #probably don't use this for now?


In [ ]:
# %%time

# feature_array = run_dict['atlas_df'][['mz','rt_peak']].values
# reference_array = msdata[['mz','rt']].values
# idx = np.isclose(feature_array[:100,None], reference_array, rtol=0.0, atol=[0.01,0.15]).all(axis=2) #order is m/z, rt, polarity
# feature_idx, reference_idx = np.where(idx)
#     # df_hits = df_hits.loc[reference_idx]
# # df_hits['original_compound_idx'] = feature_idx
# # df_hits = pd.merge(df_hits,original_peak_table_df,left_on='original_compound_idx',right_index=True,how='outer')

In [ ]:
# df = mzm.peak_height_df(metatlas_dataset)

# from scipy.spatial.distance import pdist, squareform
# df = df.reset_index(drop=True).fillna(0)
# distances = pdist(df.values, metric='euclidean')
# dist_matrix = squareform(distances)
# result = pd.DataFrame(dist_matrix)#,columns=df.columns,index=df.columns)
# result = result.stack().reset_index()
# result.columns = ['var1', 'var2','value']
# result=result.loc[(result['var1'] < result['var2']) & (result['value'] == 0 )]#< params['noise_floor']) ]

# # dd = []
# # for i,row in result.iterrows():
# #     d1 = atlas_df.loc[row.var1,'rt_max'] - atlas_df.loc[row.var1,'rt_min']
# #     d2 = atlas_df.loc[row.var2,'rt_max'] - atlas_df.loc[row.var2,'rt_min']
# #     dd.append(d1-d2)
# # result['duration'] = dd
# atlas_df['duration'] = atlas_df['rt_max'] - atlas_df['rt_min']
# result = pd.merge(result,atlas_df[['duration']],left_on='var1',right_index=True,how='left',suffixes=('1','2'))
# result = pd.merge(result,atlas_df[['duration']],left_on='var2',right_index=True,how='left',suffixes=('1','2'))

# bad_idx = result['duration2'] > result['duration1']
# bad2s = result[bad_idx]['var2'].tolist()

# bad_idx = result['duration2'] < result['duration1']
# bad1s = result[bad_idx]['var1'].tolist()

# bads = bad2s + bad1s
# bads = pd.unique(bads)
# atlas_df[~atlas_df.index.isin(bads)]
# bads
# # IF TWO FEATURES ARE IDENTICAL, KEEP THE ONE WITH THE SHORTER DURATION

In [ ]:
# mzmine_things = [
#     #potato
# #     raw data folder:
# #     /global/project/projectdirs/metatlas/raw_data/erbilgin/20161026_OE_Turnbaughswtpotato2
# #     filename structure:
# #     20161026_SK-OE_Turnbaugh_SwtPotato2_QE144_C18-USDAY32305_<pol>_<sample_type>_<??>_<??>_<??>
# #     sample_type codes:
# #     InjBl-MeOH = injection blank
# #     QC-SK160922-sopv2 = qc
# #     if it has "B" in it = extraction blank
# #     if it has a R or C = raw or cooked. The number is the bio replicate.
# #     Ignore all sample_type codes that have "H" in them; only keep the ones that have "S" in them
#                 {'polarity':'positive', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20161026_OE_Turnbaughswtpotato2', #output location
#                   'basename':'20161026_OE_Turnbaughswtpotato2_raw-v-cooked', #output file prefix string
#                  'groups':['20161026_OE_Turnbaughswtpotato2'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['_b','blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                 'min_peak_duration' : 0.025,
#                 'max_peak_duration' : 30.0,
#                 'rt_tol_perfile' : 0.015,
#                 'rt_tol_multifile' : 0.15,
#                 'min_peak_height': 1e6,
#                 'noise_floor': 3e4,
#                 'mz_tolerance': 10.0,
#                 'min_sn_ratio': 2.0,
#                 'min_rt':1.1,
#                 'max_rt':100,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_NEG','_ITSD','_H_'] #files containing these strings will be excluded
#                  },
#                 {'polarity':'negative', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20161026_OE_Turnbaughswtpotato2', #output location
#                   'basename':'20161026_OE_Turnbaughswtpotato2_raw-v-cooked', #output file prefix string
#                  'groups':['20161026_OE_Turnbaughswtpotato2'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['_b','blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                  'min_peak_duration' : 0.025,
#                 'max_peak_duration' : 30.0,
#                 'rt_tol_perfile' : 0.015,
#                 'rt_tol_multifile' : 0.15,
#                 'min_peak_height': 1e6,
#                 'noise_floor': 3e4,
#                 'mz_tolerance': 10.0,
#                 'min_sn_ratio': 2.0,
#                 'min_rt':1.1,
#                 'max_rt':100,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_POS','_ITSD','_H_'] #files containing these strings will be excluded
#                  },
    
# #     20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18
#                     {'polarity':'positive', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18', #output location
#                   'basename':'20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18', #output file prefix string
#                  'groups':['20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['waterctrl','blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e5,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_NEG'] #files containing these strings will be excluded
#                  },
#                 {'polarity':'negative', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18', #output location
#                   'basename':'20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18', #output file prefix string
#                  'groups':['20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['waterctrl','blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e5,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_POS'] #files containing these strings will be excluded
#                  },


#                         {'polarity':'positive', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18', #output location
#                   'basename':'20171116_SK_BRB_Bochner_PetriCondSpring2017_MeOH_C18', #output file prefix string
#                  'groups':['20171116_SK_BRB_Bochner_PetriCondSpring2017_MeOH_C18'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['waterctrl','blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e5,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_NEG'] #files containing these strings will be excluded
#                  },
#                 {'polarity':'negative', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18', #output location
#                   'basename':'20171116_SK_BRB_Bochner_PetriCondSpring2017_MeOH_C18', #output file prefix string
#                  'groups':['20171116_SK_BRB_Bochner_PetriCondSpring2017_MeOH_C18'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['waterctrl','blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e5,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_POS'] #files containing these strings will be excluded
#                  },
    
#                         {'polarity':'positive', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18', #output location
#                   'basename':'20171116_SK_BRB_Bochner_PetriCondSpring2017_MeOH_HIL', #output file prefix string
#                  'groups':['20171116_SK_BRB_Bochner_PetriCondSpring2017_MeOH_HIL'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['waterctrl','blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e5,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_NEG'] #files containing these strings will be excluded
#                  },
#                 {'polarity':'negative', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20171116_SK_BRB_Bochner_PetriCondSpring2017_Aq_C18', #output location
#                   'basename':'20171116_SK_BRB_Bochner_PetriCondSpring2017_MeOH_HIL', #output file prefix string
#                  'groups':['20171116_SK_BRB_Bochner_PetriCondSpring2017_MeOH_HIL'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['waterctrl','blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e5,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_POS'] #files containing these strings will be excluded
#                  },
#     #P simiae mutant lb dataset
# #     20170217_KBL_C18_RexMalm_Psimiae
#                 {'polarity':'positive', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170217_KBL_C18_RexMalm_Psimiae', #output location
#                   'basename':'20170217_KBL_C18_RexMalm_Psimiae_supernatant', #output file prefix string
#                  'groups':['20170217_KBL_C18_RexMalm_Psimiae'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_NEG','_ITSD','_pellet_'] #files containing these strings will be excluded
#                  },
#                 {'polarity':'negative', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170217_KBL_C18_RexMalm_Psimiae', #output location
#                   'basename':'20170217_KBL_C18_RexMalm_Psimiae_supernatant', #output file prefix string
#                  'groups':['20170217_KBL_C18_RexMalm_Psimiae'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl','excontrol'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_POS','_ITSD','_pellet_'] #files containing these strings will be excluded
#                  },
    
#                 #LeBoldus
#                 {'polarity':'positive', #polarity of this analysis
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/LeBoldus_innoc_and_noninnoc_poplar', #output location
#                   'basename':'20170731_KBL_C18LipidV7_PZ-KM_RootFnl', #output file prefix string
#                  'groups':['20170501_KBL_C18_15min_JL-KD_PopFun'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',#files must contain this substring % is the wildcard
#                   'file_filters':['_QC','_NEG','_ITSD'] #files containing these strings will be excluded
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/LeBoldus_innoc_and_noninnoc_poplar',
#                   'basename':'20170731_KBL_C18LipidV7_PZ-KM_RootFnl',
#                  'groups':['20170501_KBL_C18_15min_JL-KD_PopFun'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/LeBoldus_innoc_and_noninnoc_poplar',
#                   'basename':'20170503_KBL_ZHILIC_JL_PoplarFungus',
#                  'groups':['20170503_KBL_ZHILIC_JL_PoplarFungus'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/LeBoldus_innoc_and_noninnoc_poplar',
#                   'basename':'20170503_KBL_ZHILIC_JL_PoplarFungus',
#                  'groups':['20170503_KBL_ZHILIC_JL_PoplarFungus'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },
    
#                 #BRAUNI
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni',
#                   'basename':'20170919_KBL_C18_TD-DB_BrauniiLipids',
#                  'groups':['20170919_KBL_C18_TD-DB_BrauniiLipids'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni',
#                   'basename':'20170919_KBL_C18_TD-DB_BrauniiLipids',
#                  'groups':['20170919_KBL_C18_TD-DB_BrauniiLipids'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni',
#                   'basename':'20170914_KBL_C18Lipid_TD-DB_Braunii',
#                  'groups':['20170914_KBL_C18Lipid_TD-DB_Braunii'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170919_KBL_C18_TD-DB_Brauni',
#                   'basename':'20170914_KBL_C18Lipid_TD-DB_Braunii',
#                  'groups':['20170914_KBL_C18Lipid_TD-DB_Braunii'],
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },
    
#                 #Zerbe-Murphy
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZerbeMurphy',
#                   'basename':'20170728_KBL_C18_PZ-KM_RootFnl',
#                  'groups':['20170728_KBL_C18_PZ-KM_RootFnl'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZerbeMurphy',
#                   'basename':'20170728_KBL_C18_PZ-KM_RootFnl',
#                  'groups':['20170728_KBL_C18_PZ-KM_RootFnl'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZerbeMurphy',
#                   'basename':'20170731_KBL_C18LipidV7_PZ-KM_RootFnl',
#                  'groups':['20170731_KBL_C18LipidV7_PZ-KM_RootFnl'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/20170728_KBL_ZerbeMurphy',
#                   'basename':'20170731_KBL_C18LipidV7_PZ-KM_RootFnl',
#                  'groups':['20170731_KBL_C18LipidV7_PZ-KM_RootFnl'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },
    
#                 #RIPP project
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo',
#                   'basename':'20170901_KBL_C18_DP-PB_PoplarRhizo_media',
#                  'groups':['20170901_KBL_C18_DP-PB_PoplarRhizo'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD','pellet']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo',
#                   'basename':'20170901_KBL_C18_DP-PB_PoplarRhizo_media',
#                  'groups':['20170901_KBL_C18_DP-PB_PoplarRhizo'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD','pellet']
#                  },
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo',
#                   'basename':'20170901_KBL_C18_DP-PB_PoplarRhizo_pellet',
#                  'groups':['20170901_KBL_C18_DP-PB_PoplarRhizo'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD','media']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/RITT_Project_20170901_KBL_C18_DP-PB_PoplarRhizo',
#                   'basename':'20170901_KBL_C18_DP-PB_PoplarRhizo_pellet',
#                  'groups':['20170901_KBL_C18_DP-PB_PoplarRhizo'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD','media']
#                  },


#                 #Solar Panels Spain vs Berkeley
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel_Spain_vs_Berkeley',
#                   'basename':'20160824_and_20170808_C18_MP_Solar',
#                  'groups':['20160825_KBL_C18_MP_Solar_Berk','20170728_KBL_C18_MP_Solar_Spain'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel_Spain_vs_Berkeley',
#                   'basename':'20160824_and_20170808_C18_MP_Solar',
#                  'groups':['20160825_KBL_C18_MP_Solar_Berk','20170728_KBL_C18_MP_Solar_Spain'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel_Spain_vs_Berkeley',
#                   'basename':'20160824_and_20170808_C18lipid_MP_Solar',
#                  'groups':['20160824_KBL_C18Lipid_SolarPanel_MP','20170808_KBL_C18lipid_MP_Solar_Spain'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/SolarPanel_Spain_vs_Berkeley',
#                   'basename':'20160824_and_20170808_C18lipid_MP_Solar',
#                  'groups':['20160824_KBL_C18Lipid_SolarPanel_MP','20170808_KBL_C18lipid_MP_Solar_Spain'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },
    
#                 #P simiae plate flip corrected
#                 {'polarity':'positive',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/P_Simiae_plate_flip',
#                   'basename':'20170428_KBL_C18_Psimiae_DiffSubs',
#                  'groups':['20170428_KBL_C18_Psimiae_DiffSubs_FlipCorrected'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_NEG','_ITSD']
#                  },
#                 {'polarity':'negative',
#                   'basedir':'/project/projectdirs/metatlas/projects/jgi_projects/P_Simiae_plate_flip',
#                   'basename':'20170428_KBL_C18_Psimiae_DiffSubs',
#                  'groups':['20170428_KBL_C18_Psimiae_DiffSubs_FlipCorrected'], #list of folders containg datafiles
#                  'metatlas_path':'export PYTHONPATH="/global/homes/b/bpb/repos/metatlas:${PYTHONPATH}"',
#                  'blank_str':['blank','injbl','meohbl'], # lowercase strings that will identify blanks
#                  'sample_to_blank':3,#cutoff for excluding features in the blank
#                  'rt_timespan':0.5, #rt_peak must be within this amount from mzmine's rt and local minima must be +/- this amount
#                  'peak_to_valley_ratio':3,
#                  'min_intensity':1e6,
#                   'filename_substring':'%',
#                   'file_filters':['_QC','_POS','_ITSD']
#                  },

# ]


In [ ]:
# flip_file = '/global/homes/b/bpb/Downloads/20170428_KBL_RM_C18_Psimiae_PlateFlip_NewFileNames_2 (2).xlsx'
# new_dir = '/global/project/projectdirs/metatlas/raw_data/bpb/20170428_KBL_C18_Psimiae_DiffSubs_FlipCorrected'
# df = pd.read_excel(flip_file)
# for i,row in df.iterrows():
#     print(os.path.basename(row['New file name']),os.path.isfile(row['Old file name']))
#     new_loc = os.path.join(new_dir,os.path.basename(row['New file name']))
#     copyfile(row['Old file name'],new_loc)

In [ ]:
# df_super = df[[c for c in df.columns if not 'super' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'kana' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'max_intensity' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'Excontrol' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'adduct' in c]]

# df_super.set_index(['label','mz','mz_tolerance','rt_peak','rt_min','rt_max','inchi_key','detected_polarity'],inplace=True)
# df_super.columns = [''.join(c.split('_')[2:3]) if 'pellet' in c else c for c in df_super.columns ]


In [ ]:
# df_super = df[[c for c in df.columns if not 'pellet' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'kana' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'max_intensity' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'Excontrol' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'adduct' in c]]
# df_super['rt_min'] = df_super['rt_peak'] - 0.2
# df_super.set_index(['label','mz','mz_tolerance','rt_peak','rt_min','rt_max','inchi_key','detected_polarity'],inplace=True)
# df_super.columns = [''.join(c.split('_')[2:3]) if 'super' in c else c for c in df_super.columns ]


In [ ]:
# import numpy as np
# from scipy.stats import ttest_ind

# group1 = df_super[[c for c in df_super.columns if c == 'LB']]
# group2 = df_super[[c for c in df_super.columns if c == 'PS']]

# # group2 = df[df['group'] == 'GROUP2']['data'].astype(float)

# t, p = ttest_ind(group1.T, group2.T)

# stats_df = pd.DataFrame(index=df_super.index)
# stats_df['p_value'] = p
# stats_df['t_score'] = t
# stats_df['log2_fold_change'] = group2.min(axis=1).apply(lambda x: np.log2(x+1)) - \
#                                 group1.max(axis=1).apply(lambda x: np.log2(x+1)) 
# stats_df['in_control'] = group1.median(axis=1) > 1e5
# stats_df['median_control'] = group1.median(axis=1)
# stats_df['median_treatment'] = group2.median(axis=1)
# stats_df.head()


In [ ]:
# fig, ax = plt.subplots(figsize=(8,7))
# stats_df['log2_fold_change'].hist(bins=100)
# ax.set_yscale('log')
# ax.set_xlabel('LOG2 Fold Change')
# ax.set_ylabel('#features')
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(8,7))
# ax.plot(stats_df['log2_fold_change'],stats_df['p_value'],'.')
# # stats_df['fold_change'].apply(lambda x: np.log2(x+1)).hist(bins=100)
# ax.set_yscale('log')
# ax.set_xlabel('LOG2 fold change')
# ax.set_ylabel('p_value')
# plt.show()

In [ ]:
# filtered_df = stats_df[(stats_df['log2_fold_change']>7) & (stats_df['median_treatment']>1e5)]
# fig, ax = plt.subplots(figsize=(8,7))
# # color_map = {filtered_df.log2_fold_change.min():"r", filtered_df.log2_fold_change.max():"g"} 
# # z_as_colors = map(color_map.get, filtered_df.log2_fold_change) 
# sc = ax.scatter(filtered_df.index.get_level_values('mz'),
#                 filtered_df.index.get_level_values('rt_peak'),
#                 c = filtered_df.log2_fold_change)
# # stats_df['fold_change'].apply(lambda x: np.log2(x+1)).hist(bins=100)
# # ax.set_yscale('log')
# plt.colorbar(sc)
# ax.set_xlabel('mz')
# ax.set_ylabel('rt')

In [ ]:
# # (~stats_df['in_control']) & 
# filtered_df = stats_df[(stats_df['log2_fold_change']>7) & (stats_df['median_treatment']>3e5)]
# print filtered_df.shape
# # pd.options.display.precision = 5
# # pd.set_option('display.max',200)
# filtered_df.sort_values('log2_fold_change', ascending=False).head(20)

In [ ]:
# df_super.iloc[df_super.index.get_level_values('label') == '754.3994@3.75']

In [ ]:
# filtered_df.head()

In [ ]:

# # filtered_df.index.names
# df = pd.DataFrame(columns=filtered_df.index.names)
# for i,row in enumerate(filtered_df.index.get_values()):
#     for j,c in enumerate(df.columns):
#         df.loc[i,c] = row[j]
# # df.columns = 
# # filtered_df.index.get_values()
# # #.reset_index(level=[0,1,2])
# # atlas_df = atlas_df[atlas_df.columns[:6]]
# # # atlas_df.reset_index(inplace=True)
# # # atlas_df = atlas_df[[]]
# # # atlas_df.reset_index(inplace=True)
# # atlas_df.fillna('')
# # atlas_df

In [ ]:
# myAtlas = dp.make_atlas_from_spreadsheet(df,'20170406_ions_made_psimiae_in_LB_pos_c18',filetype='dataframe',
#                                        sheetname='',
#                                        polarity = 'positive',
#                                        store=True,
#                                       mz_tolerance = 10)
# atlas_df = ma_data.make_atlas_df(myAtlas)


In [ ]:

# groups = dp.select_groups_for_analysis(name = '%rexmalm_pos_super%',
#                                        most_recent = True,
#                                        remove_empty = True,#'Strain=SB214'
#                                        include_list = [], exclude_list = [],)#['QC','Blank'])


In [ ]:
# all_files = []
# for my_group in groups:
#     for my_file in my_group.items:
#         all_files.append((my_file,my_group,atlas_df,myAtlas))
        
# pool = mp.Pool(processes=min(10, len(all_files)))
# # from metatlas.helpers.metatlas_get_data_helper_fun import get_data_for_atlas_df_and_file
# metatlas_dataset = pool.map(ma_data.get_data_for_atlas_df_and_file, all_files)
# pool.close()
# pool.terminate()
# #If you're code crashes here, make sure to terminate any processes left open.


In [ ]:
# output_dir = '/global/homes/b/bpb/Downloads/psim_c18_important_pos_super/'
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
# atlas_identifications = dp.export_atlas_to_spreadsheet(myAtlas,os.path.join(output_dir,'atlas_export.csv'))
# # dp = reload(dp)
# # atlas_identifications = dp.export_atlas_to_spreadsheet(myAtlas,'%s/sheets/%s.csv'%(plot_location_label,myAtlas.name))
# peak_height = dp.make_output_dataframe(input_fname = '',input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_height' , output_loc=os.path.join(output_dir,'sheets'))
# # peak_area = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_area' , output_loc=os.path.join(output_dir,'sheets'))
# # mz_peak = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_peak' , output_loc=os.path.join(output_dir,'sheets'))
# # rt_peak = dp.make_output_dataframe(input_fname = my_file, input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [],fieldname='rt_peak' , output_loc=os.path.join(output_dir,'sheets'))
# # mz_centroid = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_centroid' , output_loc=os.path.join(output_dir,'sheets'))
# # rt_centroid = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='rt_centroid' , output_loc=os.path.join(output_dir,'sheets'))

In [ ]:
# sf = pd.DataFrame()
# for c in peak_height.columns.sort_values():
#     sf[c] = peak_height[c]
# for i,row in sf.iterrows():
#     sf.loc[i,:] = row/row.max()

In [ ]:
# cm = plt.colormaps()

# for i,c in enumerate(cm):
#     print i,c

In [ ]:
# for i,r in sf.iterrows():
#     print r.max()

In [ ]:
# # The returned object has a savefig method that should be used if you want to save the figure object without clipping the dendrograms.

# # To access the reordered row indices, use: clustergrid.dendrogram_row.reordered_ind

# # Column indices, use: clustergrid.dendrogram_col.reordered_ind

# g.dendrogram_row.reordered_ind

In [ ]:
# import seaborn as sns
# cmap = cm[148]
# # cmap = sns.light_palette(as_cmap=True)
# # cmap = sns.diverging_palette(h_neg=210, h_pos=350, s=90, l=30, as_cmap=True)
# # g  =sns.clustermap(peak_height.apply(lambda x: (x+1)**0.25),metric='correlation',figsize=(25,25),cmap=cmap)
# # g  =sns.clustermap(peak_height.apply(lambda x: np.log10(x+1)),metric='correlation',figsize=(25,25))
# g  =sns.clustermap(sf,metric='euclidean',figsize=(30,18),cmap=cmap,col_cluster=True)
# plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
# plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
# plt.show()
# # # http://seaborn.pydata.org/generated/seaborn.clustermap.html

In [ ]:
# from scipy import stats
# zscore = lambda x: (x - stats.nanmean(x)) / stats.nanstd(x)


In [ ]:
# fig, ax = plt.subplots(2,3,figsize=(20,30))
# for i in range(6):
#     plt.subplot(2,3,i+1)
#     row = peak_height.iloc[[i]]
#     c = 0
#     vals = []
#     for k,v in row.to_dict().items():
#         vals.append(v.items()[0][-1])
#     v = np.asarray(vals)
# #     v = v - np.nanmin(v)
# #     v = v / np.nanmax(v)
#     v[np.isnan(v)] = np.nanmin(v)
#     plt.plot(v,'.-')
#     plt.title(myAtlas.compound_identifications[i].name)
#     plt.gca().set_yscale('log')
# #     ax.set_yscale('log')
# plt.tight_layout()
# # for i,row in peak_height.iterrows():

    
# # peak_height.iloc[0,]